In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import scipy
from sklearn.linear_model import LogisticRegression

In [6]:
df = pd.read_csv('../Data/cat.csv')
df = df.drop('id', axis=1)
df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [7]:
dtarget = df['target']
df = df.drop('target', axis=1)
df = df.drop('nom_7', axis=1)
df = df.drop('nom_8', axis=1)
df = df.drop('nom_9', axis=1)

In [8]:
dt = pd.get_dummies(df, columns=df.columns, sparse=True)
dt = dt.sparse.to_coo()
dt = dt.tocsr()
dt.shape

(300000, 1045)

In [9]:
size = 0.8
np.random.seed(123)
ids = np.random.rand(dt.shape[0]) < size #list(np.where(np.random.rand(dt.shape[0]) < size)[0])

In [10]:
x = dt[ids]; y = dtarget[ids]
x_val = dt[~ids]; y_val = dtarget[~ids]

In [11]:
model = LogisticRegression(C=0.1, class_weight='balanced',max_iter=10000, solver='lbfgs', n_jobs=-1)
model.fit(x, y)

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
from sklearn import metrics

pred = model.predict_proba(x)
print(metrics.roc_auc_score(y, pred[:,1]))

pred = model.predict_proba(x_val)
metrics.roc_auc_score(y_val, pred[:,1])

0.7896080668908174


0.7890631616539103

## Naive base

In [13]:
dt = pd.get_dummies(df, columns=df.columns, sparse=True)
x = dt[ids]; y = dtarget[ids]
x_val = dt[~ids]; y_val = dtarget[~ids]

In [14]:
p = x[y==1].sum(0) + 1
q = x[y==0].sum(0) + 1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(q)/len(q))

In [16]:
pred = x[r.index] @ r.T + b
print(metrics.roc_auc_score(y, pred))

pred = x_val[r.index] @ r.T + b
metrics.roc_auc_score(y_val, pred)

0.7892608054869308


0.7883608513340878

In [28]:
#x_nb = x.multiply(r)
model = LogisticRegression(C=0.2, class_weight='balanced',max_iter=10000, solver='lbfgs', n_jobs=-1)
model.fit(x_nb, y)

LogisticRegression(C=0.2, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
pred = model.predict_proba(x_nb)
print(metrics.roc_auc_score(y, pred[:,1]))

0.7876293011157469


In [30]:
pred = model.predict_proba(x_val.multiply(r))
metrics.roc_auc_score(y_val, pred[:,1])

0.7878459813992685